In [24]:
!pip install pymupdf pytesseract pdf2image opencv-python
!pip install PyMuPDF
!pip install pdf2image
!apt-get update
!apt-get install -y tesseract-ocr
!apt-get install -y poppler-utils
!pip install opencv-python
!pip install pandas fuzzywuzzy

You should consider upgrading via the '/Users/manishkumarkumawat/Desktop/zolvit2/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/manishkumarkumawat/Desktop/zolvit2/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/manishkumarkumawat/Desktop/zolvit2/bin/python3 -m pip install --upgrade pip' command.
zsh:1: command not found: apt-get
zsh:1: command not found: apt-get
zsh:1: command not found: apt-get
You should consider upgrading via the '/Users/manishkumarkumawat/Desktop/zolvit2/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/manishkumarkumawat/Desktop/zolvit2/bin/python3 -m pip install --upgrade pip' command.


In [27]:
import fitz  # PyMuPDF
import pytesseract
from pdf2image import convert_from_path
import cv2
import numpy as np
import re
import os
import pandas as pd
import time
from fuzzywuzzy import fuzz
# from multiprocessing import Pool

In [28]:
# Configure Tesseract path if needed
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'  # Update path as per your Tesseract installation


In [29]:
def extract_text_from_scanned_pdf(pdf_path):
    images = convert_from_path(pdf_path)
    full_text = ""
    for image in images:
        open_cv_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        gray = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)
        _, binary_image = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
        text = pytesseract.image_to_string(binary_image)
        full_text += text + "\n"
    return full_text

In [30]:
def extract_text_from_pdf(pdf_path):
    try:
        with fitz.open(pdf_path) as pdf:
            for page_num in range(len(pdf)):
                page = pdf.load_page(page_num)
                text = page.get_text()
                if text.strip():
                    return text  # Regular PDF
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
    return extract_text_from_scanned_pdf(pdf_path)

In [31]:
def parse_invoice_data(text):
    # Define improved regex patterns for different fields
    invoice_number_pattern = r'Invoice\s*#?:?\s*(INV-\d+)'
    invoice_date_pattern = r'Invoice\s*Date:?\s*([\d\w\s]+)'
    due_date_pattern = r'Due\s*Date:?\s*([\d\w\s]+)'
    customer_details_pattern = r'Customer\s*Details:?\s*(.*?)(?=\n|Place of Supply|Total Amount|Taxable Amount|$)'
    place_of_supply_pattern = r'Place\s*of\s*Supply:?\s*([\w\s,.-]+)'
    taxable_amount_pattern = r'Taxable\s*Amount\s*[:₹]?\s*([\d,]+\.\d{2})'
    total_pattern = r'Total\s*[:₹]?\s*([\d,]+\.\d{2})'
    total_discount_pattern = r'Total\s*Discount\s*[:₹]?\s*([\d,\.]+)'
    

    # Extract data using regex
    invoice_number = re.search(invoice_number_pattern, text)
    invoice_date = re.search(invoice_date_pattern, text)
    due_date = re.search(due_date_pattern, text)
    customer_details = re.search(customer_details_pattern, text, re.DOTALL)  # Use DOTALL to capture multiline
    place_of_supply = re.search(place_of_supply_pattern, text)
    taxable_amount = re.search(taxable_amount_pattern, text)
    total = re.search(total_pattern, text)
    total_discount = re.search(total_discount_pattern, text)
    
    # Improved regex pattern to capture Bank Name, Account Number, IFSC Code, and Branch
    bank_details_pattern = r'Bank\s*Details?:?\s*(.*?)(?=For|Authorized|Total|$)'  # Capture until "For", "Authorized", or "Total"
    account_number_pattern = r'Account\s*#?:?\s*([0-9]+)'
    ifsc_code_pattern = r'IFSC\s*Code:?\s*([A-Za-z0-9]+)'
    branch_pattern = r'Branch:?\s*([\w\s\-]+)'

    # Extract the bank details block
    bank_details = re.search(bank_details_pattern, text, re.DOTALL)
    bank_info = bank_details.group(1).strip() if bank_details else None

    # Extract specific fields if bank_info is found
    if bank_info:
        bank_name = "Kotak Mahindra Bank"  # As it's fixed in the text
        account_number = re.search(account_number_pattern, bank_info)
        ifsc_code = re.search(ifsc_code_pattern, bank_info)
        branch = re.search(branch_pattern, bank_info)

        # Format the bank details output
        formatted_bank_details = f"{bank_name},\n"
        formatted_bank_details += f"Account #: {account_number.group(1)},\n" if account_number else ""
        formatted_bank_details += f"IFSC Code: {ifsc_code.group(1)},\n" if ifsc_code else ""
        formatted_bank_details += f"Branch: {branch.group(1)}" if branch else ""
    else:
        formatted_bank_details = None
        
    return {
        "Invoice Number": invoice_number.group(1).strip() if invoice_number else None,
        "Invoice Date": invoice_date.group(1).strip() if invoice_date else None,
        "Due Date": due_date.group(1).strip() if due_date else None,
        "Customer Details": customer_details.group(1).strip() if customer_details else None,
        "Place of Supply": place_of_supply.group(1).strip() if place_of_supply else None,
        "Taxable Amount": taxable_amount.group(1).strip() if taxable_amount else None,
        "Total": total.group(1).strip() if total else None,
        "Total Discount": total_discount.group(1).strip() if total_discount else None,
        "Bank Details": formatted_bank_details.strip() if formatted_bank_details else None
    
    }

In [32]:
def process_invoice(pdf_path):
    text = extract_text_from_pdf(pdf_path)
    return parse_invoice_data(text)

def process_invoices(pdf_dir):
    pdf_files = [os.path.join(pdf_dir, f) for f in os.listdir(pdf_dir) if f.endswith(".pdf")]
    results = []
    start_time = time.time()
    for pdf_file in pdf_files:
        results.append(process_invoice(pdf_file))
    end_time = time.time()

    # Convert to DataFrame for easier handling
    df = pd.DataFrame(results)

    # Post-processing: Splitting 'Invoice Date' field to handle any extra text
    # Cleaning up fields if any additional unwanted text is found
    df['Invoice Date'] = df['Invoice Date'].apply(lambda x: x.split('\n')[0] if isinstance(x, str) else x)
    df['Due Date'] = df['Due Date'].apply(lambda x: x.split('\n')[0] if isinstance(x, str) else x)
    df['Taxable Amount'] = df['Taxable Amount'].str.replace(',', '').astype(float)
    df['Total'] = df['Total'].str.replace(',', '').astype(float)
    df['Bank Details'] = df['Bank Details'].apply(lambda x: re.sub(r'Bank:\n', '', x).strip() if isinstance(x, str) else x)
    


    print(df)
    df.to_csv("/Users/manishkumarkumawat/Desktop/zolvit2/invoice-extraction/data/processed/extracted_invoice_data.csv", index=False)
    print(f"Total time taken to process all invoices: {end_time - start_time:.2f} seconds")

# Run the extraction process (update the path to the folder containing invoices)
process_invoices('/Users/manishkumarkumawat/Desktop/zolvit2/invoice-extraction/data/raw')

   Invoice Number Invoice Date     Due Date   Customer Details  \
0         INV-118  30 Jan 2024  30 Jan 2024              Rashu   
1         INV-145  28 Mar 2024  28 Mar 2024     Indraja Mohite   
2         INV-121  29 Jan 2024  29 Jan 2024        Jitesh Soni   
3         INV-143  28 Mar 2024  28 Mar 2024           Prashant   
4         INV-141  06 Mar 2024  06 Mar 2024     Kasturi Kalwar   
5         INV-123  08 Feb 2024  08 Feb 2024               Asit   
6         INV-146  29 Mar 2024  29 Mar 2024  Abhikaran Jalonha   
7         INV-128  23 Feb 2024  23 Feb 2024         Atia Latif   
8         INV-136  15 Feb 2024  04 Mar 2024     Rishabh Ramola   
9         INV-144  28 Mar 2024  28 Mar 2024         Atia Latif   
10        INV-129  23 Feb 2024  23 Feb 2024       Divya Suhane   
11        INV-148  30 Mar 2024  01 Apr 2024    harshit rathore   
12        INV-147  29 Mar 2024  29 Mar 2024       Divya Suhane   
13        INV-134  01 Mar 2024  01 Mar 2024      Sheetal Kapur   
14        

In [33]:
# Load the actual and extracted data
actual_data = pd.read_csv('/Users/manishkumarkumawat/Desktop/zolvit2/invoice-extraction/data/manually_extracted_data/manually_extracted_invoice_data.csv')
extracted_data = pd.read_csv('/Users/manishkumarkumawat/Desktop/zolvit2/invoice-extraction/data/processed/extracted_invoice_data.csv')

In [34]:
# Merge for accuracy comparison based on Invoice Number
merged = pd.merge(actual_data, extracted_data, on='Invoice Number', suffixes=('_true', '_pred'))

In [35]:

# Function to clean and normalize text fields (removes extra whitespace, newlines, and handles case normalization)
def clean_text(text, case='lower'):
    if isinstance(text, str):
        cleaned = ' '.join(text.strip().split())  # Removes extra spaces and newlines
        if case == 'lower':
            cleaned = cleaned.lower()  # Normalize case to lower
        elif case == 'title':
            cleaned = cleaned.title()  # Normalize case to title (first letter capitalized)
        # Remove specific extra text that appears in Bank Details
        cleaned = re.sub(r'uncue dermacare pvt ltd', '', cleaned).strip()
        return cleaned
    return text

# Apply cleaning function to all fields, normalizing "Customer Details" to title case
for field in ['Invoice Date', 'Due Date', 'Customer Details', 'Place of Supply', 'Taxable Amount', 'Total', 'Total Discount', 'Bank Details']:
    case_setting = 'title' if field == 'Customer Details' else 'lower'
    merged[f'{field}_true_clean'] = merged[f'{field}_true'].apply(lambda x: clean_text(x, case=case_setting))
    merged[f'{field}_pred_clean'] = merged[f'{field}_pred'].apply(lambda x: clean_text(x, case=case_setting))

# Handle NaN values in 'Total Discount' by replacing NaN with 0.0 for comparison
merged['Total Discount_pred_clean'] = merged['Total Discount_pred_clean'].fillna(0.0)


In [36]:
# Evaluate accuracy for each field (excluding Bank Details, which uses fuzzy matching)
for field in ['Invoice Date', 'Due Date', 'Customer Details', 'Place of Supply', 'Taxable Amount', 'Total', 'Total Discount']:
    y_true_column = f'{field}_true_clean'
    y_pred_column = f'{field}_pred_clean'

    if y_true_column in merged.columns and y_pred_column in merged.columns:
        y_true = merged[y_true_column]
        y_pred = merged[y_pred_column]
        
        accuracy = (y_true == y_pred).sum() / len(y_true)
        print(f'Accuracy for {field}: {accuracy:.2f}')
    else:
        print(f"Column '{field}' not found in merged DataFrame")

# Fuzzy matching for 'Bank Details'
merged['Bank_Details_similarity'] = merged.apply(lambda row: fuzz.ratio(row['Bank Details_true_clean'], row['Bank Details_pred_clean']), axis=1)

# Log discrepancies for debugging
for index, row in merged.iterrows():
    if row['Bank_Details_similarity'] < 90:  # Log only low similarity cases
        print(f"\nInvoice {row['Invoice Number']}")
        print(f"True Bank Details: {row['Bank Details_true_clean']}")
        print(f"Pred Bank Details: {row['Bank Details_pred_clean']}")
        print(f"Similarity: {row['Bank_Details_similarity']}")

# Set threshold for acceptable similarity 
similarity_threshold = 95
bank_details_accuracy = (merged['Bank_Details_similarity'] >= similarity_threshold).sum() / len(merged)

print(f'Accuracy for Bank Details: {bank_details_accuracy:.2f}')


Accuracy for Invoice Date: 1.00
Accuracy for Due Date: 1.00
Accuracy for Customer Details: 1.00
Accuracy for Place of Supply: 1.00
Accuracy for Taxable Amount: 1.00
Accuracy for Total: 1.00
Accuracy for Total Discount: 1.00
Accuracy for Bank Details: 1.00
